In [2]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

# Load the EMNIST dataset
train, test = tfds.load('mnist', split='train[:75%]', shuffle_files=True), tfds.load('mnist', split='test[:25%]', shuffle_files=True)
assert isinstance(train, tf.data.Dataset)
# (train_images, train_labels), (test_images, test_labels) = emnist.load_data('letters')

# Normalize the images
train_images = train / 255.0
test_images = test / 255.0

# Define a function to create a TensorFlow Dataset from numpy arrays
def create_tf_dataset(images, labels, batch_size=64, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(images))
    dataset = dataset.batch(batch_size)
    return dataset

# Split the training dataset into train and validation sets
train_size = int(0.8 * len(train_images))
val_size = len(train_images) - train_size

train_dataset = create_tf_dataset(train_images[:train_size], train_labels[:train_size])
val_dataset = create_tf_dataset(train_images[train_size:], train_labels[train_size:], shuffle=False)
test_dataset = create_tf_dataset(test_images, test_labels, shuffle=False)

# Data augmentation using ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# Define the CNN architecture using class implementation
class EMNISTCNN(Model):
    def __init__(self):
        super(EMNISTCNN, self).__init__()
        self.conv1 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.pool1 = layers.MaxPooling2D((2, 2))
        self.conv2 = layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.pool2 = layers.MaxPooling2D((2, 2))
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(128, activation='relu')
        self.bn3 = layers.BatchNormalization()
        self.dropout = layers.Dropout(0.5)
        self.fc2 = layers.Dense(47, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.bn3(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Instantiate the model
model = EMNISTCNN()

# Define the optimizer with gradient clipping
optimizer = optimizers.Adam(learning_rate=0.001, decay=1e-4)
loss_object = losses.SparseCategoricalCrossentropy(from_logits=False)

# Training function with gradient clipping
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    clipped_gradients = [tf.clip_by_norm(g, 1.0) for g in gradients]
    optimizer.apply_gradients(zip(clipped_gradients, model.trainable_variables))
    return loss

# Training function
def train_model(model, train_dataset, val_dataset, epochs):
    history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': []}
    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')
        # Training loop
        for images, labels in train_dataset:
            loss = train_step(images, labels)

        # Validation loop
        val_loss = 0
        val_accuracy = 0
        for val_images, val_labels in val_dataset:
            val_predictions = model(val_images, training=False)
            val_loss += loss_object(val_labels, val_predictions)
            val_accuracy += tf.reduce_sum(tf.cast(tf.equal(tf.argmax(val_predictions, axis=1), val_labels), tf.float32))
        val_loss /= len(val_dataset)
        val_accuracy /= len(val_dataset) * 64

        history['loss'].append(loss.numpy())
        history['val_loss'].append(val_loss.numpy())
        history['accuracy'].append(val_accuracy.numpy())
        history['val_accuracy'].append(val_accuracy.numpy())
        print(f'Loss: {loss.numpy()}, Val Loss: {val_loss.numpy()}, Val Accuracy: {val_accuracy.numpy() * 100:.2f}%')
    return history

# Test function
def test_model(model, test_dataset):
    test_loss = 0
    test_accuracy = 0
    for test_images, test_labels in test_dataset:
        test_predictions = model(test_images, training=False)
        test_loss += loss_object(test_labels, test_predictions)
        test_accuracy += tf.reduce_sum(tf.cast(tf.equal(tf.argmax(test_predictions, axis=1), test_labels), tf.float32))
    test_loss /= len(test_dataset)
    test_accuracy /= len(test_dataset) * 64
    return test_loss.numpy(), test_accuracy.numpy()

# Plot metrics
def plot_metrics(history):
    plt.figure(figsize=(16, 12))

    plt.subplot(2, 2, 1)
    plt.plot(history['loss'], 'b', label='Training loss')
    plt.plot(history['val_loss'], 'g', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(2, 2, 2)
    plt.plot(history['accuracy'], 'b', label='Training accuracy')
    plt.plot(history['val_accuracy'], 'g', label='Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

def main():
    epochs = 10
    history = train_model(model, train_dataset, val_dataset, epochs)
    test_loss, test_acc = test_model(model, test_dataset)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc * 100:.2f}%')
    plot_metrics(history)

if __name__ == '__main__':
    main()

TypeError: unsupported operand type(s) for /: '_PrefetchDataset' and 'float'